## DAA CIA 2 - Rishi - 21110277

##### Importing modules

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

##### Loading the data

In [24]:
df = pd.read_csv(r"C:\Users\ASUS\Downloads\Bank_Personal_Loan_Modelling.csv")

In [25]:
X = df.drop(["Personal Loan", "ID"], axis = 1)
y = df["Personal Loan"]

##### Data preprocessing

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.25, random_state=0)

In [27]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## PyTorch

In [28]:
class TrainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)

In [29]:
train_data = TrainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
class TestData(Dataset):
    def __init__(self, X_data):
        self.X_data = X_data

    def __getitem__(self, index):
        return self.X_data[index]

    def __len__ (self):
        return len(self.X_data)

In [30]:
test_data = TestData(torch.FloatTensor(X_test))

In [31]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [32]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()

        self.layer_1 = nn.Linear(12, 12)
        self.layer_2 = nn.Linear(12, 8)
        self.layer_out = nn.Linear(8, 1)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)

        return x

In [33]:
model = BinaryClassification()
print(model)

BinaryClassification(
  (layer_1): Linear(in_features=12, out_features=12, bias=True)
  (layer_2): Linear(in_features=12, out_features=8, bias=True)
  (layer_out): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
)


In [34]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

In [35]:
def binary_acc(y_pred, y_test):
    y_pred = torch.round(torch.sigmoid(y_pred))
    acc = (y_pred == y_test).sum().float() / y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [36]:
epoch = 50
for e in range(1, epoch+1):
    epoch_loss = 0
    epoch_acc = 0
    for X, y in train_loader:
        optimizer.zero_grad()

        y_pred = model(X)

        loss = criterion(y_pred, y.unsqueeze(1))
        acc = binary_acc(y_pred, y.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()


    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.39533 | Acc: 90.136
Epoch 002: | Loss: 0.27845 | Acc: 90.119
Epoch 003: | Loss: 0.22523 | Acc: 90.153
Epoch 004: | Loss: 0.18683 | Acc: 90.136
Epoch 005: | Loss: 0.15023 | Acc: 93.220
Epoch 006: | Loss: 0.12264 | Acc: 95.847
Epoch 007: | Loss: 0.10705 | Acc: 96.271
Epoch 008: | Loss: 0.09779 | Acc: 96.542
Epoch 009: | Loss: 0.09251 | Acc: 96.898
Epoch 010: | Loss: 0.08752 | Acc: 96.814
Epoch 011: | Loss: 0.08331 | Acc: 96.983
Epoch 012: | Loss: 0.08137 | Acc: 97.068
Epoch 013: | Loss: 0.07793 | Acc: 97.186
Epoch 014: | Loss: 0.07641 | Acc: 97.288
Epoch 015: | Loss: 0.07483 | Acc: 97.102
Epoch 016: | Loss: 0.07195 | Acc: 97.407
Epoch 017: | Loss: 0.07024 | Acc: 97.407
Epoch 018: | Loss: 0.06914 | Acc: 97.458
Epoch 019: | Loss: 0.06769 | Acc: 97.525
Epoch 020: | Loss: 0.06642 | Acc: 97.644
Epoch 021: | Loss: 0.06504 | Acc: 97.644
Epoch 022: | Loss: 0.06345 | Acc: 97.627
Epoch 023: | Loss: 0.06375 | Acc: 97.695
Epoch 024: | Loss: 0.06263 | Acc: 97.661
Epoch 025: | Los

In [37]:
y_pred_torch = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_torch.append(y_pred_tag.cpu().numpy())


In [38]:
y_pred_torch = [a.squeeze().tolist() for a in y_pred_torch]

## Accuracy Score

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_torch))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1142
           1       0.96      0.83      0.89       108

    accuracy                           0.98      1250
   macro avg       0.97      0.91      0.94      1250
weighted avg       0.98      0.98      0.98      1250



## Genetic Algorithm

In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
def split(df,label):
    X_tr, X_te, Y_tr, Y_te = train_test_split(df, label, test_size=0.25, random_state=42)
    return X_tr, X_te, Y_tr, Y_te

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score

classifiers = ['LinearSVM', 'RadialSVM', 
               'Logistic',  'RandomForest', 
               'AdaBoost',  'DecisionTree', 
               'KNeighbors','GradientBoosting']

models = [svm.SVC(kernel='linear'),
          svm.SVC(kernel='rbf'),
          LogisticRegression(max_iter = 1000),
          RandomForestClassifier(n_estimators=200, random_state=0),
          AdaBoostClassifier(random_state = 0),
          DecisionTreeClassifier(random_state=0),
          KNeighborsClassifier(),
          GradientBoostingClassifier(random_state=0)]


def acc_score(df,label):
    Score = pd.DataFrame({"Classifier":classifiers})
    j = 0
    acc = []
    X_train,X_test,Y_train,Y_test = split(df,label)
    for i in models:
        model = i
        model.fit(X_train,Y_train)
        predictions = model.predict(X_test)
        acc.append(accuracy_score(Y_test,predictions))
        j = j+1     
    Score["Accuracy"] = acc
    Score.sort_values(by="Accuracy", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score

def plot(score,x,y,c = "b"):
    gen = [1,2,3,4,5]
    plt.figure(figsize=(6,4))
    ax = sns.pointplot(x=gen, y=score,color = c )
    ax.set(xlabel="Generation", ylabel="Accuracy")
    ax.set(ylim=(x,y))

In [41]:
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)     
        chromosome[:int(0.3*n_feat)]=False             
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population


def fitness_score(population):
    scores = []
    for chromosome in population:
        logmodel.fit(X_train.iloc[:,chromosome],Y_train)         
        predictions = logmodel.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(Y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)                                    
    return list(scores[inds][::-1]), list(population[inds,:][::-1]) 


def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen


def crossover(pop_after_sel):
    pop_nextgen = pop_after_sel
    for i in range(0,len(pop_after_sel),2):
        new_par = []
        child_1 , child_2 = pop_nextgen[i] , pop_nextgen[i+1]
        new_par = np.concatenate((child_1[:len(child_1)//2],child_2[len(child_1)//2:]))
        pop_nextgen.append(new_par)
    return pop_nextgen


def mutation(pop_after_cross,mutation_rate,n_feat):   
    mutation_range = int(mutation_rate*n_feat)
    pop_next_gen = []
    for n in range(0,len(pop_after_cross)):
        chromo = pop_after_cross[n]
        rand_posi = [] 
        for i in range(0,mutation_range):
            pos = randint(0,n_feat-1)
            rand_posi.append(pos)
        for j in rand_posi:
            chromo[j] = not chromo[j]  
        pop_next_gen.append(chromo)
    return pop_next_gen

def generations(df,label,size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, Y_train, Y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print('Best score in generation',i+1,':',scores[:1])  #2
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate,n_feat)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

In [44]:
data_bc = pd.read_csv(r"C:\Users\ASUS\Downloads\Bank_Personal_Loan_Modelling.csv")
label_bc = data_bc["Personal Loan"]

In [45]:
display(data_bc.head())

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [46]:
score1 = acc_score(data_bc,label_bc)
score1

,Classifier,Accuracy
0,RandomForest,1.0000
1,AdaBoost,1.0000
2,DecisionTree,1.0000
3,GradientBoosting,1.0000
4,LinearSVM,0.9072
5,Logistic,0.9000
6,KNeighbors,0.8936
7,RadialSVM,0.8928


## Cultural Algorithm

In [47]:
import random
import numpy as np

input_size = 10
hidden_size = 20
output_size = 5
learning_rate = 0.01

population_size = 50
cultural_pool_size = 10
mutation_rate = 0.1

def generate_population(size):
    population = []
    for i in range(size):
        input_hidden = np.random.randn(input_size, hidden_size)
        hidden_output = np.random.randn(hidden_size, output_size)
        population.append((input_hidden, hidden_output))
    return population

def evaluate_fitness(individual):
    input_hidden, hidden_output = individual
    return accuracy

def sort_population(population):
    population.sort(key=lambda individual: evaluate_fitness(individual), reverse=True)

def update_cultural_pool(population, pool_size):
    sort_population(population)
    return population[:pool_size]

def mutate(parent):
    input_hidden, hidden_output = parent
    input_hidden += np.random.randn(input_size, hidden_size) * mutation_rate
    hidden_output += np.random.randn(hidden_size, output_size) * mutation_rate
    return (input_hidden, hidden_output)

def cultural_algorithm():
    population = generate_population(population_size)
    cultural_pool = update_cultural_pool(population, cultural_pool_size)
    for i in range(100):
        parent = random.choice(cultural_pool)
        child = mutate(parent)
        child_fitness = evaluate_fitness(child)
        # Replace a random individual in the population with the child if its fitness is better
        random_index = random.randint(0, population_size - 1)
        if child_fitness > evaluate_fitness(population[random_index]):
            population[random_index] = child
        # Update the cultural pool with the best individuals from the population
        cultural_pool = update_cultural_pool(population, cultural_pool_size)
    sort_population(population)
    return population[0]

In [48]:
generate_population(14)

[(array([[ 2.61224588e-01, -1.49644234e-01,  2.41626798e-01,
          -9.73904739e-01,  2.65567214e+00, -9.03778749e-01,
          -1.41815286e+00, -7.41340590e-01,  1.10718127e+00,
          -1.48229774e-01,  7.30368170e-02, -3.07338457e-01,
          -4.43510447e-01, -1.66093008e+00,  6.60561795e-01,
           5.22841093e-01,  1.93173296e+00, -6.40645162e-01,
          -6.86010898e-01,  1.16657149e+00],
         [ 7.93566426e-01,  1.50682136e-04,  8.75268565e-01,
           1.03089540e+00, -1.27823754e+00, -9.95673276e-01,
          -5.27218722e-01,  1.01099905e+00,  1.32486105e-01,
          -7.78949961e-01,  1.17414648e-02, -6.23877198e-01,
          -5.67861750e-01,  1.25572444e+00,  1.21549630e-01,
          -8.34574529e-01,  6.91310243e-02,  2.77246208e-01,
           1.51632546e+00,  2.47337684e-01],
         [ 4.38310658e-01, -2.34803915e-02, -7.45516380e-01,
          -2.86096961e-03, -1.25633655e+00, -6.95369774e-02,
          -1.43814595e+00, -6.27121094e-01,  1.15729529e

In [52]:
score2 = acc_score(data_bc,label_bc)
score2

,Classifier,Accuracy
0,RandomForest,1.0000
1,AdaBoost,1.0000
2,DecisionTree,1.0000
3,GradientBoosting,1.0000
4,LinearSVM,0.9072
5,Logistic,0.9000
6,KNeighbors,0.8936
7,RadialSVM,0.8928


# Swarm Optimisation

In [49]:
import random
import math    
import copy    
import sys     
# ------------------------------------
def show_vector(vector):
  for i in range(len(vector)):
    if i % 14 == 0:
      print("\n", end="")
    if vector[i] >= 0.0:
      print(' ', end="")
    print("%.4f" % vector[i], end="") 
    print(" ", end="")
  print("\n")
def error(position):
  err = 0.0
  for i in range(len(position)):
    xi = position[i]
    err += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
  return err
# ------------------------------------
class Particle:
  def __init__(self, dim, minx, maxx, seed):
    self.rnd = random.Random(seed)
    self.position = [0.0 for i in range(dim)]
    self.velocity = [0.0 for i in range(dim)]
    self.best_part_pos = [0.0 for i in range(dim)]
    for i in range(dim):
      self.position[i] = ((maxx - minx) *
        self.rnd.random() + minx)
      self.velocity[i] = ((maxx - minx) *
        self.rnd.random() + minx)
    self.error = error(self.position) # curr error
    self.best_part_pos = copy.copy(self.position) 
    self.best_part_err = self.error # best error
def Solve(max_epochs, n, dim, minx, maxx):
  rnd = random.Random(0)
  # create n random particles
  swarm = [Particle(dim, minx, maxx, i) for i in range(n)] 
  best_swarm_pos = [0.0 for i in range(dim)] # not necess.
  best_swarm_err = sys.float_info.max # swarm best
  for i in range(n): # check each particle
    if swarm[i].error < best_swarm_err:
      best_swarm_err = swarm[i].error
      best_swarm_pos = copy.copy(swarm[i].position) 
  epoch = 0
  w = 0.729    # inertia
  c1 = 1.49445 # cognitive (particle)
  c2 = 1.49445 # social (swarm)
  while epoch < max_epochs:
    
    if epoch % 2 == 0 and epoch > 1:
      print("Epoch = " + str(epoch) +
        " best error = %.30f" % best_swarm_err)
    for i in range(n): # process each particle
      
      # compute new velocity of curr particle
      for k in range(dim): 
        r1 = rnd.random()    # randomizations
        r2 = rnd.random()
    
        swarm[i].velocity[k] = ( (w * swarm[i].velocity[k]) +
          (c1 * r1 * (swarm[i].best_part_pos[k] -
          swarm[i].position[k])) +  
          (c2 * r2 * (best_swarm_pos[k] -
          swarm[i].position[k])) )  
        if swarm[i].velocity[k] < minx:
          swarm[i].velocity[k] = minx
        elif swarm[i].velocity[k] > maxx:
          swarm[i].velocity[k] = maxx
      # compute new position using new velocity
      for k in range(dim): 
        swarm[i].position[k] += swarm[i].velocity[k]
  
      # compute error of new position
      swarm[i].error = error(swarm[i].position)
      # is new position a new best for the particle?
      if swarm[i].error < swarm[i].best_part_err:
        swarm[i].best_part_err = swarm[i].error
        swarm[i].best_part_pos = copy.copy(swarm[i].position)
      # is new position a new best overall?
      if swarm[i].error < best_swarm_err:
        best_swarm_err = swarm[i].error
        best_swarm_pos = copy.copy(swarm[i].position)
    
    # for-each particle
    epoch += 1
  # while
  return best_swarm_pos
# end Solve
dim = 14
print("Goal is to solve Rastrigin's function in " +
 str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
  print("0, ", end="")
print("0)")
num_particles = 50
max_epochs = 100
print("Setting num_particles = " + str(num_particles))
print("Setting max_epochs    = " + str(max_epochs))
print("\nStarting PSO algorithm\n")
best_position = Solve(max_epochs, num_particles,
 dim, -10.0, 10.0)
print("\nPSO completed\n")
print("\nBest solution found:")
show_vector(best_position)
err = error(best_position)
print("Error of best solution = %.6f" % err)

Goal is to solve Rastrigin's function in 14 variables
Function has known min = 0.0 at (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Setting num_particles = 50
Setting max_epochs    = 100

Starting PSO algorithm

Epoch = 2 best error = 230.376018951605487927736248821020
Epoch = 4 best error = 230.376018951605487927736248821020
Epoch = 6 best error = 230.376018951605487927736248821020
Epoch = 8 best error = 166.936107003028951112355571240187
Epoch = 10 best error = 148.149222885737827937191468663514
Epoch = 12 best error = 148.149222885737827937191468663514
Epoch = 14 best error = 148.149222885737827937191468663514
Epoch = 16 best error = 124.300145182445675118287908844650
Epoch = 18 best error = 115.094066411355569812258181627840
Epoch = 20 best error = 115.094066411355569812258181627840
Epoch = 22 best error = 111.968589832358503599607502110302
Epoch = 24 best error = 111.968589832358503599607502110302
Epoch = 26 best error = 111.968589832358503599607502110302
Epoch = 28 best error = 111.

## Ant Colony Optimisation

In [50]:
import numpy as np

class AntColonyOptimizer:
    def __init__(self, num_ants, num_iterations, alpha, beta, rho, q, nn, X_train, y_train):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.nn = nn
        self.X_train = X_train
        self.y_train = y_train
        self.num_weights = nn.get_num_weights()

    def optimize(self):
        pheromone_matrix = np.ones((self.num_weights,))
        best_weight = None
        best_loss = np.inf

        for i in range(self.num_iterations):
            for j in range(self.num_ants):
                weights = self.generate_weights(pheromone_matrix)
                loss = self.nn.compute_loss(weights, self.X_train, self.y_train)
                self.update_pheromone_matrix(pheromone_matrix, weights, loss)
                if loss < best_loss:
                    best_weight = weights
                    best_loss = loss

        return best_weight

    def generate_weights(self, pheromone_matrix):
        weights = np.zeros((self.num_weights,))
        for i in range(self.num_weights):
            p = np.power(pheromone_matrix[i], self.alpha)
            q = np.power(self.nn.weights[i], self.beta)
            prob = p*q / np.sum(p*q)
            weights[i] = np.random.choice(self.nn.weight_range, p=prob)
        return weights

    def update_pheromone_matrix(self, pheromone_matrix, weights, loss):
        for i in range(self.num_weights):
            pheromone_matrix[i] *= (1 - self.rho)
            if weights[i] == self.nn.weights[i]:
                pheromone_matrix[i] += self.q / loss


In [58]:
import numpy as np

NV = 10

def formPheremoneMatrix(graph):
    
    nv = len(graph)
    pheremones = np.ones((nv,nv))
        
    return pheremones
           
def chooseNextVertex(graph, pheremones, currPos):
    graph = 1/graph
    denominator = np.dot(graph[currPos], pheremones[currPos])
    numerator = graph[currPos] * pheremones[currPos]
    
    probabilities = numerator/denominator
    
    
    rouletteWheel = np.cumsum(probabilities)
    
    rouletteBall = np.random.random()
    
    nextVertex = np.where(rouletteWheel >= rouletteBall)[0][0]
    
    return nextVertex

def traverse(graph, pheremones, start, end):
    curr = start
    path = [curr]
    cost = 0
    prev = start
    
    while curr != end:
        nextVertex = chooseNextVertex(graph, pheremones, curr)
        
        while nextVertex == prev:
            nextVertex = chooseNextVertex(graph, pheremones, curr)
        
        cost += graph[curr][nextVertex]
        path += [nextVertex]
        prev = curr
        curr = nextVertex
        
    
    return path, cost
    
    

def releaseGeneration(graph, pheremones, start, end, size = 10):
    paths = []
    costs = []
    for i in range(size):
        p, c = traverse(graph, pheremones, start, end)
        paths += [p]
        costs += [c]
    costs = np.array(costs)    
    
    return paths, costs
    
def updatePheremones(paths, costs, pheremones, decay = 0):
    pheremones = (1-decay)*pheremones
    
    costs = 1/costs
    for p in range(len(paths)):
        path = paths[p]
        for i in range(len(path) - 1):
            pheremones[path[i]][path[i+1]] += costs[p]
    
    return pheremones
            

def generateProblem(size, density):
    graph = np.full((size, size), np.inf)
    
    for i in range(len(graph)):
        for j in range(i, len(graph)):
            if np.random.random() < density:
                if i!=j:
                    w = np.random.randint(1,20)
                    graph[i][j] = w
                    graph[j][i] = w
                    
    return graph
#%%
graph = generateProblem(NV, 0.5)

#%%
ph = formPheremoneMatrix(graph)
print(graph)

#%%
gen = 0
SIZE = 100

for i in range(100):
    p,c = releaseGeneration(graph, ph, 0, 7, SIZE)
    ph = updatePheremones(p, c, ph, decay = 0)
    gen += 1

    print("=",gen,"=")    
    c = np.array(c)
    unique, counts = np.unique(c, return_counts=True)
    print(unique)
    print(counts)
    
    if len(np.where(counts > SIZE//2)[0]) == 1:
        break

[[inf inf inf 14. 17.  1.  6. inf inf inf]
 [inf inf inf 16. inf 18.  5. 14.  2. inf]
 [inf inf inf  4. inf  7. inf  6. inf 12.]
 [14. 16.  4. inf 14. inf inf inf 16. inf]
 [17. inf inf 14. inf 18. inf 18. inf inf]
 [ 1. 18.  7. inf 18. inf  7. inf 12. 13.]
 [ 6.  5. inf inf inf  7. inf  7. 16.  9.]
 [inf 14.  6. inf 18. inf  7. inf inf 19.]
 [inf  2. inf 16. inf 12. 16. inf inf inf]
 [inf inf 12. inf inf 13.  9. 19. inf inf]]
= 1 =
[ 13.  14.  15.  26.  27.  29.  30.  32.  33.  35.  36.  37.  38.  39.
  41.  42.  43.  44.  46.  47.  50.  51.  52.  53.  54.  56.  59.  61.
  62.  63.  65.  72.  73.  74.  78.  79.  80.  81.  83.  89.  94.  98.
 101. 104. 113. 115. 119. 128. 130. 136. 137. 143. 148. 149. 168. 170.
 192. 194. 208. 254. 344. 364. 431.]
[4 8 2 1 2 1 2 2 5 1 3 2 1 1 3 3 2 2 1 2 2 1 1 1 1 1 1 1 1 1 1 3 2 1 1 1 2
 1 2 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1]
= 2 =
[ 13.  14.  15.  24.  27.  28.  29.  30.  32.  33.  35.  36.  37.  39.
  40.  42.  43.  44.  47.  48.  49. 